In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import sys
from sklearn.metrics import accuracy_score, classification_report

In [2]:
# determine what to run my tensors on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# my one hot encoders
input_size = 223
# this is a parameter 
hidden_size = 300
# output : num of classes
num_classes = 11
# num of trials (epochs)
epochs = 10
# Batch size = ? 
batch_size = 10
# learning_rate
lr = 0.01

# num_layers in RNN default is 1 (increasing layers improve result but worsen the time)

In [100]:
tokens_tensor = torch.load("tokens.pt")
# taking a batch cuz suffering from memory issues
tokens = tokens_tensor[2250000:]
del tokens_tensor

C:\Users\abdo_\AppData\Local\Temp\ipykernel_22984\3488547841.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tokens_tensor = torch.load("tokens.pt")


In [101]:
print(tokens.shape)

torch.Size([206446, 25, 223])


In [102]:
labels_tensor = torch.load("labels.pt")
labels = labels_tensor[2250000:]
del labels_tensor

C:\Users\abdo_\AppData\Local\Temp\ipykernel_22984\2355988855.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  labels_tensor = torch.load("labels.pt")


In [8]:
# we can use the DataSet class from pytorch to facilitate 
# batch divisions and data preparation
class SimpleDataset(Dataset):
    def __init__(self, input_indices, labels):
        self.input_indices = input_indices
        self.labels = labels

    def __len__(self):
        return len(self.input_indices)

    def __getitem__(self, idx):
        return self.input_indices[idx], self.labels[idx]

In [103]:
tokens = tokens.type(torch.float32)
labels = labels.type(torch.float32)
data = SimpleDataset(tokens, labels)
data = DataLoader(data, batch_size=10)

In [104]:
del tokens
del labels

In [3]:
class RNN(nn.Module):
    def __init__(self, input_size, num_classes, hidden_size):
        super(RNN,self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        # batch_first = True means that batch is the first dimension
        # shape : batch_first, seq, input_size
        self.lstm = nn.LSTM(input_size=input_size,hidden_size=hidden_size, batch_first=True).to(device)
        # linear layer : from hidden RNN to Output
        self.fc = nn.Linear(hidden_size, num_classes).to(device)

    def forward(self, input):
        # for LSTM we need initial tensor state + initial hidden state
        # where initial tensor state is called (cell)
        c_0 = torch.zeros(1,input.size(0),self.hidden_size).to(device)
        h_0 = torch.zeros(1,input.size(0),self.hidden_size).to(device)
        # output of self.rnn : out feature, hidden_state(n)
        out, _ = self.lstm(input,(h_0,c_0))
        # size of output = batch, seq_length, hidden_size
        out = self.fc(out)
        probabilities = torch.softmax(out, dim=2)
        return probabilities



In [8]:

model = RNN(input_size,num_classes,hidden_size)

loss_criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(),lr)

In [97]:

for epoch in range(10):
    
    for input_tensors, label_tensors in data:
        
        input_tensors = input_tensors.to(device)
        
        label_tensors = label_tensors.to(device)

        out_tensor = model(input_tensors)
    
        loss = loss_criterion(out_tensor,label_tensors)

        optimizer.zero_grad()

        loss.backward()
        
        optimizer.step()

    print(f"epoch {epoch}:, loss ={loss.item()}")

epoch 0:, loss =2.4191884994506836
epoch 1:, loss =2.4191880226135254
epoch 2:, loss =2.4191863536834717
epoch 3:, loss =2.4191854000091553
epoch 4:, loss =2.419184923171997
epoch 5:, loss =2.419184684753418
epoch 6:, loss =2.419184446334839
epoch 7:, loss =2.419184446334839
epoch 8:, loss =2.4191842079162598
epoch 9:, loss =2.4191842079162598


In [98]:
del data

In [99]:
torch.save(model.state_dict(),"model.pth")

In [114]:
# since our model requires_grad if we made a test it will put the equations in the grad graph
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in data:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)

            # Get predictions
            preds = torch.argmax(outputs, dim=2)  # Shape: (batch_size, seq_length, features) dim = 2 : features

            # Flatten predictions and labels
            all_preds.extend(preds.cpu().numpy().flatten())
            all_labels.extend(torch.argmax(labels,dim=2).cpu().numpy().flatten())


In [115]:
accuracy = accuracy_score(all_labels, all_preds)
report = classification_report(all_labels, all_preds)
print("Accuracy:", accuracy)
print("Classification Report:\n", report)

c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMet

Accuracy: 0.45730583300233474
Classification Report:
               precision    recall  f1-score   support

           0       0.56      0.01      0.01   2105318
           1       0.69      0.98      0.81    139323
           2       0.00      0.00      0.00    125000
           3       1.00      1.00      1.00    825784
           4       0.58      1.00      0.73    226629
           5       1.00      1.00      1.00    206446
           6       0.00      0.00      0.00         0
           8       1.00      0.45      0.62    746493
           9       1.00      0.78      0.88    786157
          10       0.00      0.00      0.00         0

    accuracy                           0.46   5161150
   macro avg       0.58      0.52      0.51   5161150
weighted avg       0.77      0.46      0.48   5161150



c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [5]:
list = ["i", "want", "one", "cheeseburg", "pizza","and","one","can","of","pepsi"]
tensor_tokens_test = torch.load("token_test.pt")
tensor_labels = torch.load("label_tensor.pt")

C:\Users\abdo_\AppData\Local\Temp\ipykernel_23704\695977007.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor_tokens_test = torch.load("token_test.pt")
C:\Users\abd

In [9]:
model.load_state_dict(torch.load("model.pth"))

C:\Users\abdo_\AppData\Local\Temp\ipykernel_23704\1722216506.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>

In [10]:
tensor_tokens_test = tensor_tokens_test.reshape(-1,10,input_size)
tensor_labels = tensor_labels.reshape(-1,10,num_classes)

In [18]:
tensor_labels= tensor_labels.type(torch.float32).to(device)
tensor_tokens_test= tensor_tokens_test.type(torch.float32).to(device)

In [13]:
model.eval()

RNN(
  (lstm): LSTM(223, 300, batch_first=True)
  (fc): Linear(in_features=300, out_features=11, bias=True)
)

In [28]:
output = model(tensor_tokens_test)

In [29]:
tensor_labels = torch.argmax(tensor_labels,dim=2)

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [31]:
output = torch.argmax(output,dim=2)

In [32]:
print(tensor_labels)
print(output )

tensor([[3, 3, 4, 9, 5, 3, 4, 0, 6, 1]], device='cuda:0')
tensor([[3, 3, 4, 4, 5, 3, 4, 0, 6, 1]], device='cuda:0')


tensor([[3, 3, 4, 4, 5, 3, 4, 0, 6, 1]], device='cuda:0')
